# Exemplo ventás 1
En primeiro lugar iniciamos a sesión como nos casos anteriores:


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession.builder \
    .appName("Windows") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

print("Versión: ",spark.version)


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a2f1420d-64dd-4f78-8350-84e8627746be;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

Versión:  3.5.7


Neste exemplo imos ler, de novo, os datos desde un *socket*. Antes de nada póñese en marcha co seguinte comando:
```bash
nc -lk 9999
```

O seguinte paso é poñer en marcha o *stream* de lectura, nesta ocasión activando a opción *includeTimestamp*.


In [2]:
df_lineas = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", "9999") \
    .option('includeTimestamp', 'true')\
    .load()

df_lineas.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



Do mesmo xeito que no exemplo orixinal do *socket*, imos facer un *wordcount*; a diferenza é que neste caso inclúese o *timestamp*.


In [3]:
from pyspark.sql.functions import explode, split
df_palabras = df_lineas.select(
    explode(split(df_lineas.value, ' ')).alias('palabra'),
    df_lineas.timestamp)

Créase unha xanela fixa de dous minutos. Isto agrupará os datos, palabra e reconto, por períodos fixos de tempo.


In [4]:
from pyspark.sql.functions import window
windowed_counts = df_palabras.groupBy(
    window(df_palabras.timestamp, "2 minutes"), df_palabras.palabra
).count().orderBy('window')

windowed_counts.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- palabra: string (nullable = false)
 |-- count: long (nullable = false)



Realízase a consulta indicando como *sink* a consola. 

In [6]:
query = windowed_counts \
          .writeStream \
          .outputMode("complete") \
          .format("console") \
          .queryName("consulta1") \
          .option("truncate","false") \
          .start()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-------+-----+
|window|palabra|count|
+------+-------+-----+
+------+-------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|y       |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|hasta   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|el      |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|probando|2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|no      |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|cuando  |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mazo    |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|dando   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|con     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|se      |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mola    |1    |
+------------------------------------------+--------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|y       |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|hasta   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|nada    |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|el      |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mas     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|probando|2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|no      |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|cuando  |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mazo    |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|dando   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|con     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|se      |2    |
|{2

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|y       |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|hasta   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|nada    |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|el      |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mas     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|probando|2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|no      |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|cuando  |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mazo    |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|dando   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|con     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|se      |2    |
|{2

-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|y       |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|hasta   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|nada    |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|el      |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mas     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|probando|2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|no      |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|cuando  |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mazo    |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|dando   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|con     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|se      |2    |
|{2

-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+--------+-----+
|window                                    |palabra |count|
+------------------------------------------+--------+-----+
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|y       |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|hasta   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|nada    |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|el      |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mas     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|probando|2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|no      |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|cuando  |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|mazo    |2    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|dando   |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|con     |1    |
|{2026-02-09 18:20:00, 2026-02-09 18:22:00}|se      |2    |
|{2

> Como se pode observar, as agregacións vanse realizando por ventás de dous minutos.